In [35]:
import os
import pandas as pd
import numpy as np 
import datetime as dt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from api_credentials import client_id, client_secret

# Initialize the Spotify client with client credentials for public data access
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Read in the AU Daily 200 Spotify Chart  
Manually downloaded CSV from https://charts.spotify.com/charts/view/regional-au-daily/latest

In [36]:
# Load the CSV file
file_path = 'raw_data/regional-au-daily-2024-02-05.csv'
df = pd.read_csv(file_path)

# Extract the date from the file name
file_name = os.path.basename(file_path)  # Gets 'regional-au-daily-2024-02-05.csv'
date_str = file_name.split('-')[-3:]  # Splits the file name and takes the last 3 elements ['2024', '02', '05.csv']
date_str = '-'.join(date_str)  # Joins them back into '2024-02-05.csv'
date = date_str.split('.')[0]  # Removes the file extension, resulting in '2024-02-05'

# Convert the date string to a datetime object
date_datetime = pd.to_datetime(date)

# Format the datetime object to a string in 'DD-MM-YYYY' format
formatted_date = date_datetime.strftime('%d-%m-%Y')

# Add the formatted date as a new column to the DataFrame
df['date'] = formatted_date

Extract each tracks unique id for API call 

In [37]:
track_ids = df['uri'].str[14:]
df['track_id'] = df['uri'].str[14:]
df.drop(columns='uri', inplace=True)

Call Spotify API to retrieve `popularity_score` for each of the top 200 tracks

In [38]:
# Assuming `track_ids` is your Pandas Series of track IDs as shown
# Convert the Series to a list for processing
track_ids_list = track_ids.tolist()

# Spotify's `tracks` endpoint can take multiple track IDs at once,
# but to avoid potential rate limits or payload size issues,
# it's a good idea to process them in batches.
# Here, we define a batch size (e.g., 50 tracks per request)
batch_size = 50

# Initialize lists to store the retrieved data
artists = []
track_names = []
popularity_scores = []

for i in range(0, len(track_ids_list), batch_size):
    batch = track_ids_list[i:i+batch_size]
    tracks_info = sp.tracks(batch)
    for track in tracks_info['tracks']:
        # Some tracks might not be available, leading to `None` entries
        if track:
            # Extracting the primary artist name
            artist_name = track['artists'][0]['name'] if track['artists'] else 'N/A'
            artists.append(artist_name)
            
            # Extracting the track name
            track_names.append(track['name'])
            
            # Extracting the popularity score
            popularity_scores.append(track['popularity'])

# Now that you have all the data, you can put it into a DataFrame
popularity_df = pd.DataFrame({
    'artist_name': artists,
    'track_name': track_names,
    'popularity_score': popularity_scores
})

# Show the DataFrame to verify the results
popularity_df.head()


,artist_name,track_name,popularity_score
0,Noah Kahan,Stick Season,97
1,Jack Harlow,Lovin On Me,99
2,Benson Boone,Beautiful Things,93
3,Tate McRae,greedy,100
4,Taylor Swift,Cruel Summer,99


In [39]:
df['popularity_score'] = popularity_df['popularity_score']

In [40]:
# Reorder columns
df = df[['date', 'rank', 'artist_names', 'track_name', 'track_id', 'source', 'peak_rank', 'previous_rank', 'days_on_chart', 'streams', 'popularity_score']]

# Rename columns
df.rename(columns={
    'source': 'label',
    'streams': 'daily_streams',
}, inplace=True)

In [41]:
df.head()

,date,rank,artist_names,track_name,track_id,label,peak_rank,previous_rank,days_on_chart,daily_streams,popularity_score
0,05-02-2024,1,Noah Kahan,Stick Season,0mflMxspEfB0VbI1kyLiAv,Mercury Records/Republic Records,1,1,123,333388,97
1,05-02-2024,2,Jack Harlow,Lovin On Me,4xhsWYTOGcal8zt0J161CU,Generation Now/Atlantic,1,2,88,272882,99
2,05-02-2024,3,Benson Boone,Beautiful Things,6tNQ70jh4OwmPGpYy6R2o9,"Night Street Records, Inc./Warner Records Inc.",3,3,18,267772,93
3,05-02-2024,4,Tate McRae,greedy,3rUGC1vUpkDG9CZFHMur1t,RCA Records Label,1,4,144,232303,100
4,05-02-2024,5,Taylor Swift,Cruel Summer,1BxfuPKGuaTgP7aM0Bbdwr,Taylor Swift,1,5,366,220866,99


In [42]:
df['previous_rank'].unique()

array([  1,   2,   3,   4,   5,   6,   9,   7,   8,  12,  10,  16,  14,
        11,  13,  15,  20,  18,  17,  19,  21,  22,  25,  26,  23,  24,
        28,  34,  27,  32,  29,  56,  30,  33,  36,  41,  37,  40,  31,
        38,  39,  57,  50,  35,  49,  43,  44,  42,  46,  52,  48,  47,
        45,  51,  53,  58,  55,  70,  54,  60,  62,  61,  76,  66,  71,
        83,  59,  63,  73,  64,  65,  91,  72,  69,  80,  77,  68,  75,
        84,  87,  82,  78,  79,  81,  86,  67,  89,  74,  88,  93,  85,
        94,  98,  92,  99,  95, 109,  97,  90,  96, 106, 100, 166, 101,
       102, 108, 113, 122, 111, 157, 135, 112, 110, 107, 118, 116, 127,
       104, 114, 128, 187, 130, 103, 138, 126, 105, 115, 117, 161, 158,
       120, 134, 131, 121, 151, 132, 119, 139, 142, 168, 185, 149, 156,
       123, 188, 137, 146, 140, 125, 147, 124, 145,  -1, 150, 174, 160,
       129, 162, 153, 175, 190, 133, 136, 159, 152, 154, 173, 163, 186,
       171, 176, 191, 179, 183, 148, 165, 141, 143, 155, 180, 16

Retrieve additional data from Kworb.net
This will give us additional features/columns like:  
 - `count_at_peak`
 - `rank_movement`
 - `streams+`
 - `7day_streams`
 - `7day_streams_movement` 
 - `total_streams`

In [43]:
# Read the HTML table into a DataFrame
kworb_df = pd.read_html('https://kworb.net/spotify/country/au_daily.html')[0]

# Get the current date minus 2 days to account for data lag
current_date_minus_2 = dt.datetime.now().date() - dt.timedelta(days=2)

# Add the current date as a new column to the DataFrame
kworb_df['DATE'] = current_date_minus_2

In [14]:
kworb_df = kworb_df.rename(columns={
    'Pos' : 'rank',
    'P+': 'rank_change',
    'Artist and Title' : 'artist_title',
    'Days' : 'days_on_chart',
    'Pk' : 'peak_rank',
    '(x?)' : 'days_at_peak',
    'Streams' : 'daily_streams',
    'Streams+' : 'daily_streams_change',
    '7Day' : '7Day_streams',
    '7Day+' : '7Day_streams_change',
    'Total' : 'total_streams',
})

In [16]:
# Selecting specific columns to view
selected_columns_df = kworb_df[['artist_title', 'rank_change', 'days_at_peak', 'daily_streams_change', '7Day_streams', '7Day_streams_change', 'total_streams']]

# Display the DataFrame with the selected columns
selected_columns_df['rank_change'].unique()


array(['=', '+2', '-1', '+4', '+1', '-3', '-2', '+3', '+6', '+24', '+5',
       '-8', '+15', '+7', '-9', '-6', '-5', '+12', '+13', '+17', '+19',
       '-4', '-19', '-14', '+63', '+14', '+47', '-7', '+10', '+8', '+66',
       '-20', '-21', '-12', '-11', '+32', '+28', '-13', '+16', '-18',
       '+44', '+43', '-24', '-27', 'RE', '-28', '+29', '-29', '+18',
       '-37', '-36', '-26', '-16', '-43', '-32', '-17'], dtype=object)

Need to change "=" to "0", remove "+", and handle "RE"

In [17]:
# Remove "+" and ensure "=" is replaced with "0"
kworb_df['rank_change'] = kworb_df['rank_change'].str.replace('+', '').str.replace('=', '0', regex=False)

# Attempt to convert the column to numeric without coercing errors to identify potential non-numeric values
try:
    kworb_df['rank_change'] = pd.to_numeric(kworb_df['rank_change'])
    print("Conversion successful. No NaN values detected.")
except ValueError as e:
    print("Conversion failed due to non-numeric values:", e)

# Optionally, inspect the column for any remaining non-numeric values or anomalies
# This step is just for verification and won't change the DataFrame
non_numeric = kworb_df[pd.to_numeric(kworb_df['rank_change'], errors='coerce').isna()]
if not non_numeric.empty:
    print("Non-numeric values found:\n", non_numeric)
else:
    print("No non-numeric values found.")

Conversion failed due to non-numeric values: Unable to parse string "RE" at position 152
Non-numeric values found:
      rank rank_change                                       artist_title  \
152   153          RE                                Travis Scott - FE!N   
162   163          RE  21 Savage - née-nah (w/ Travis Scott, Metro Bo...   
179   180          RE                       Taylor Swift - Lavender Haze   
184   185          RE                              The Kid LAROI - BLEED   
192   193          RE         Florence + The Machine - Dog Days Are Over   
193   194          RE                             Jason Mraz - I'm Yours   
194   195          RE         Kendrick Lamar - Money Trees (w/ Jay Rock)   
196   197          RE                        Kylie Minogue - Padam Padam   
198   199          RE                               J. Cole - Wet Dreamz   
199   200          RE           Kanye West - Gold Digger (w/ Jamie Foxx)   

     days_on_chart  peak_rank days_at_peak  dai

/var/folders/px/x65fp4pn31s5cv4gsbh_3cl80000gn/T/ipykernel_2467/641918597.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  kworb_df['rank_change'] = kworb_df['rank_change'].str.replace('+', '').str.replace('=', '0', regex=False)


In [18]:
# Extract numeric values from 'COUNT_AT_PEAK'
kworb_df['days_at_peak'] = kworb_df['days_at_peak'].str.extract('(\d+)', expand=False)

# Convert to numeric, allowing NaNs to remain
kworb_df['days_at_peak'] = pd.to_numeric(kworb_df['days_at_peak'], errors='coerce')

In [19]:
columns_to_keep = kworb_df[['artist_title', 'rank_change', 'days_at_peak', 'daily_streams_change', '7Day_streams', '7Day_streams_change', 'total_streams']]
columns_to_keep.head(3)

,artist_title,rank_change,days_at_peak,daily_streams_change,7Day_streams,7Day_streams_change,total_streams
0,Noah Kahan - Stick Season,0,10.0,8284.0,2376830,-13316,21011273
1,Jack Harlow - Lovin On Me,0,72.0,1522.0,2070240,-45409,28217939
2,Benson Boone - Beautiful Things,0,3.0,28872.0,1611385,91911,3005801


In [20]:
print(f"Spotify Daily 200 columns:", df.columns.tolist())
print(f"Kworb Daily 200 columns:", kworb_df.columns.tolist())

Spotify Daily 200 columns: ['date', 'rank', 'artist_names', 'track_name', 'track_id', 'label', 'peak_rank', 'previous_rank', 'days_on_chart', 'daily_streams', 'popularity_score']
Kworb Daily 200 columns: ['rank', 'rank_change', 'artist_title', 'days_on_chart', 'peak_rank', 'days_at_peak', 'daily_streams', 'daily_streams_change', '7Day_streams', '7Day_streams_change', 'total_streams', 'DATE']


In [21]:
df['rank_change'] = kworb_df['rank_change']
df['days_at_peak'] = kworb_df['days_at_peak']
df['daily_streams_change'] = kworb_df['daily_streams_change']
df['7Day_streams'] = kworb_df['7Day_streams']
df['7Day_streams_change'] = kworb_df['7Day_streams_change']
df['total_streams'] = kworb_df['total_streams']


In [22]:
df.columns.tolist()

['date',
 'rank',
 'artist_names',
 'track_name',
 'track_id',
 'label',
 'peak_rank',
 'previous_rank',
 'days_on_chart',
 'daily_streams',
 'popularity_score',
 'rank_change',
 'days_at_peak',
 'daily_streams_change',
 '7Day_streams',
 '7Day_streams_change',
 'total_streams']

In [23]:
columns_order =  [
    'date',
    'artist_names',
    'track_name',
    'rank',
    'previous_rank',
    'rank_change',
    'peak_rank',
    'days_at_peak',
    'days_on_chart',
    'popularity_score',
    'daily_streams',
    'daily_streams_change',
    '7Day_streams',
    '7Day_streams_change',
    'total_streams',
    'label',
    'track_id',
]

df = df[columns_order]

In [24]:
df.head()

,date,artist_names,track_name,rank,previous_rank,rank_change,peak_rank,days_at_peak,days_on_chart,popularity_score,daily_streams,daily_streams_change,7Day_streams,7Day_streams_change,total_streams,label,track_id
0,05-02-2024,Noah Kahan,Stick Season,1,1,0,1,10.0,123,97,333388,8284.0,2376830,-13316,21011273,Mercury Records/Republic Records,0mflMxspEfB0VbI1kyLiAv
1,05-02-2024,Jack Harlow,Lovin On Me,2,2,0,1,72.0,88,99,272882,1522.0,2070240,-45409,28217939,Generation Now/Atlantic,4xhsWYTOGcal8zt0J161CU
2,05-02-2024,Benson Boone,Beautiful Things,3,3,0,3,3.0,18,93,267772,28872.0,1611385,91911,3005801,"Night Street Records, Inc./Warner Records Inc.",6tNQ70jh4OwmPGpYy6R2o9
3,05-02-2024,Tate McRae,greedy,4,4,0,1,9.0,144,100,232303,1031.0,1654249,-4706,32729268,RCA Records Label,3rUGC1vUpkDG9CZFHMur1t
4,05-02-2024,Taylor Swift,Cruel Summer,5,5,0,1,1.0,366,99,220866,-2740.0,1560404,1221,51425313,Taylor Swift,1BxfuPKGuaTgP7aM0Bbdwr


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  200 non-null    object 
 1   artist_names          200 non-null    object 
 2   track_name            200 non-null    object 
 3   rank                  200 non-null    int64  
 4   previous_rank         200 non-null    int64  
 5   rank_change           200 non-null    object 
 6   peak_rank             200 non-null    int64  
 7   days_at_peak          44 non-null     float64
 8   days_on_chart         200 non-null    int64  
 9   popularity_score      200 non-null    int64  
 10  daily_streams         200 non-null    int64  
 11  daily_streams_change  190 non-null    float64
 12  7Day_streams          200 non-null    int64  
 13  7Day_streams_change   200 non-null    int64  
 14  total_streams         200 non-null    int64  
 15  label                 2